In [2]:
import requests
import json
import subprocess

# Estructura component

El component ha de poder executar un codi agnòstic. Es pot començar simplificant el problema fent que aquest codi agnòstic està dins d'un arxiu .py.  
Aquest arxiu .py estarà dins d'un docker.  
El docker estarà montat amb una base de fastapi.  
La crida al component, hauria d'aixecar el docker, amb uns paràmetres d'entrada. El docker executaria el codi .py, i automàticament enviaria el resultat de l'execució de tornada mitjançant fastapi  

Patrón de diseño Command Pattern

# Component V1

docker build -t componente-v1.0 -f Dockerfile .

docker run -e SCRIPT_PATH=/path/to/script.py -e ARGS="arg1 arg2 arg3" -e URL=http://api-access-manager.com/path componente-v1.0



docker run -it --rm mi-imagen python /tests/componente_tests.py

"calculos.py" debe enviar por stdout los resultados del cálculo en formato json (o str en su defecto).



# Crides

In [8]:
resultado = {"data": {"a" : 1}}
url = "http://localhost:8080/run"
headers = {"Content-Type": "application/json"}
response = requests.post(url, headers=headers, data=json.dumps(resultado))
print(response)
print(response.text)

<Response [200]>
{"status":1000}


In [132]:
url = "http://localhost:8080/results"
headers = {"Content-Type": "application/json"}
resultado =  { "a" : 1 }
response = requests.post(url, headers=headers, data=json.dumps(resultado))
json.dumps(resultado)

'{"a": 1}'

In [118]:
docker_image_name = "componente-v1.0"
script_path = "calculos.py"
args = "arg1 arg2 arg3"
url = "http://host.docker.internal:8080/results"
docker_command = ["docker", "run", "-e", f"SCRIPT_PATH={script_path}", "-e", f'ARGS="{args}"', "-e", f'URL={url}', docker_image_name]
process = subprocess.run(docker_command)
process

CompletedProcess(args=['docker', 'run', '-e', 'SCRIPT_PATH=calculos.py', '-e', 'ARGS="arg1 arg2 arg3"', '-e', 'URL=http://host.docker.internal:8080/results', 'componente-v1.0'], returncode=0)

In [34]:
docker_image_name = "componente-v1.0"
script_path = "calculos.py"
args = "arg1 arg2 arg3"
url = "http://host.docker.internal:8080/results"
docker_command = f'docker run -e SCRIPT_PATH={script_path} -e ARGS="{args}" -e URL={url} {docker_image_name}'
process = subprocess.run(docker_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,text=True)

'{"a": 1}'

In [46]:
script_path = "..\src\componente_v1\calculos.py"
try:
    with open(script_path, 'r') as file:
        print('El archivo calculos.py es accesible')
except FileNotFoundError:
    print('El archivo calculos.py no es accesible')

El archivo calculos.py es accesible


# Howto


* Construir imagen  
docker build -t componente-v1.0 -f Dockerfile .

* Para abrir el access manager   
python access_manager_simulation.py




* Para utilizar terminal dentro del contenedor (principalmente debugguear)  
docker run -it componente-v1.0 bash


In [4]:
script_name = "script_1_calculadora.py"
script_arguments = "2 3"

data = {"script_path": script_name, "arguments": script_arguments}
run_script_url = "http://localhost:8080/run"
headers = {"Content-Type": "application/json"}
response = requests.post(run_script_url, headers=headers, json=data)
print(response, response.text)

<Response [202]> {"status":"started","PID":5560}


# Modificacions  

* No s'executarà codi aliè.  
* Dos approaches, T2.3 i T2.4
* No s'executarà un docker per procés, els processos menys intensos seràn scripts a l'entorn
  que s'executaràn. Els més intensos sí que es faràn en un contenidor apart.


## Jofre: Preguntar cas d'us?  
  * Qui és?
  * Què ens ha proporcionat?
  * Quin context necessito per formular una pregunta més coherent informativa que "Com ho utilitzaries?"
## Diagrama de sqüència: Com faràn les dades? Per on passaràn?
  * Com faràn les dades?
  * Per on passaràn?
  * Visio?


# Component v2


Para construir el contenedor:
> docker build -t componente-v2 -f Dockerfile .  

Para correr el contenedor:  
> docker run -p 8000:8000 componente-v2  
 
Si quiero tener la unidad montada en /datasets para ver el resultado  
> docker run -p 8000:8000 -v C:\Users\ian.palacin\Desktop\projects\IMPETUS\src\componente_v2\datasets:/app/datasets componente-v2

De esta forma se ejecuta el comando ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"], que deja corriendo una api en el contenedor, esta será la responsable de recibir órdenes. La api está en main.py, desde allí se orquestará la:
* Recepción de órdenes des del Acess Manager (de momento simulado)
* Ejecución del proceso
* Envío de la salida del proceso de vuelta al Access Manager


### Development

Component
> uvicorn main:app --host 0.0.0.0 --port 8000 --reload  

Clean procedure  
>python procedures\clean_procedure.py datasets\racha_max_manresa.csv RachaMax

# INTERFICIE

In [1]:
import requests
import json
import pandas as pd
import plotly.graph_objects as go

In [48]:
# Vull processar les meves dades amb el procés de data cleaning
payload = {"data": {"a" : 1}}
url = "http://127.0.0.1:8080/run"
headers = {"Content-Type": "application/json"}
payload = { 
    "script_name": "clean_procedure.py",
    "args": ["datasets/racha_max_manresa.csv", "RachaMax"]
    
}
response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response)
print(response.text)

<Response [500]>
Internal Server Error


In [8]:
df_racha_raw = pd.read_csv( "..\src\componente_v2\datasets\\racha_max_manresa.csv")
df_racha_raw = df_racha_raw[['RachaMax']]
# df_racha_processed = pd.read_csv( "..\src\componente_v2\datasets\\racha_max_manresa_20230809_110038.csv")
df_racha_processed = pd.read_csv( "..\src\componente_v2\datasets\\racha_max_manresa_20230809_110038.csv")

In [ ]:
trace_raw = go.Scatter(x=df_racha_raw.index, y=df_racha_raw['RachaMax'], mode='lines', name='Raw')
trace_processed = go.Scatter(x=df_racha_processed.index, y=df_racha_processed['RachaMax'], mode='lines', name='Processed')
fig = go.Figure(data=[trace_raw, trace_processed])
fig.show()

# Lab

In [39]:
import pandas as pd
df = pd.read_excel( r"..\src\componente_v2\datasets\viento_manresa.xlsx" )
df = df[['FECHA', 'Racha Max (Km/h)']]
df.to_csv(r"..\src\componente_v2\datasets\racha_max_manresa.csv")

